Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

---
Problem 1a
----------

Logistic regression with L2 regularization

---

In [17]:
# PREPARATION STEP

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  # Added the L2 regularization here. The 0.002 value is the beta value
  # This beta value is purely arbitrary, I chose it because the accuracy seems the highest
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 0.002 * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [18]:
# EXECUTION STEP

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 24.973557
Minibatch accuracy: 7.0%
Validation accuracy: 11.3%
Minibatch loss at step 500: 3.015771
Minibatch accuracy: 78.9%
Validation accuracy: 56.9%
Minibatch loss at step 1000: 1.189450
Minibatch accuracy: 82.0%
Validation accuracy: 62.2%
Minibatch loss at step 1500: 0.738853
Minibatch accuracy: 89.1%
Validation accuracy: 64.7%
Minibatch loss at step 2000: 0.658983
Minibatch accuracy: 81.2%
Validation accuracy: 65.3%
Minibatch loss at step 2500: 0.727992
Minibatch accuracy: 83.6%
Validation accuracy: 64.8%
Minibatch loss at step 3000: 0.583065
Minibatch accuracy: 85.9%
Validation accuracy: 65.3%
Test accuracy: 88.8%


---
Problem 1b
----------

Neural network with L2 regularization

---

In [36]:
# This tutorial helps a lot: https://www.tensorflow.org/versions/v0.6.0/tutorials/mnist/tf/index.html
# PREPARATION STEP
hidden_nodes = 1024
batch_size = 300
beta = 0.002

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Set hidden layer
    hl_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hl_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hl_weights) + hl_biases)
    print(hidden_layer)
    
    # Set output layer
    ol_weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    print(ol_weights.get_shape())
    ol_biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation
    logits = tf.matmul(hidden_layer, ol_weights) + ol_biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    loss = loss + beta * (tf.nn.l2_loss(hl_weights) + tf.nn.l2_loss(ol_weights))
    
    # Optimizer - for finding the minimum of the loss using gradient descent
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)  
    
    # Predictions for the training, validation and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hl_weights) + hl_biases), ol_weights) + ol_biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hl_weights) + hl_biases),ol_weights)+ ol_biases)

Tensor("Relu:0", shape=TensorShape([Dimension(300), Dimension(1024)]), dtype=float32)
TensorShape([Dimension(1024), Dimension(10)])


In [40]:
# EXECUTION STEP
num_steps = 10001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ('Initialized')
    for step in xrange(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
           [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1056.627686
Minibatch accuracy: 11.0%
Validation accuracy: 24.4%
Minibatch loss at step 500: 230.231964
Minibatch accuracy: 79.0%
Validation accuracy: 61.1%
Minibatch loss at step 1000: 83.315727
Minibatch accuracy: 83.3%
Validation accuracy: 65.8%
Minibatch loss at step 1500: 30.762358
Minibatch accuracy: 84.7%
Validation accuracy: 71.0%
Minibatch loss at step 2000: 11.569753
Minibatch accuracy: 87.0%
Validation accuracy: 73.1%
Minibatch loss at step 2500: 4.566609
Minibatch accuracy: 87.7%
Validation accuracy: 74.3%
Minibatch loss at step 3000: 2.042636
Minibatch accuracy: 87.3%
Validation accuracy: 75.0%
Minibatch loss at step 3500: 1.060551
Minibatch accuracy: 88.7%
Validation accuracy: 75.0%
Minibatch loss at step 4000: 0.767768
Minibatch accuracy: 88.0%
Validation accuracy: 75.2%
Minibatch loss at step 4500: 0.552370
Minibatch accuracy: 91.0%
Validation accuracy: 75.6%
Minibatch loss at step 5000: 0.516656
Minibatch accuracy: 89.7%
Validation

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [24]:
# This tutorial helps a lot: https://www.tensorflow.org/versions/v0.6.0/tutorials/mnist/tf/index.html
# PREPARATION STEP
hidden_nodes = 1024
batch_size = 10
beta = 0.002

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Set hidden layer
    hl_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hl_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hl_weights) + hl_biases)
    print(hidden_layer)
    
    # Set output layer
    ol_weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    print(ol_weights.get_shape())
    ol_biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation
    logits = tf.matmul(hidden_layer, ol_weights) + ol_biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    loss = loss + beta * (tf.nn.l2_loss(hl_weights) + tf.nn.l2_loss(ol_weights))
    
    # Optimizer - for finding the minimum of the loss using gradient descent
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)  
    
    # Predictions for the training, validation and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hl_weights) + hl_biases), ol_weights) + ol_biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hl_weights) + hl_biases),ol_weights)+ ol_biases)

Tensor("Relu:0", shape=TensorShape([Dimension(10), Dimension(1024)]), dtype=float32)
TensorShape([Dimension(1024), Dimension(10)])


In [25]:
# EXECUTION STEP
num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ('Initialized')
    for step in xrange(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
           [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 863.855835
Minibatch accuracy: 20.0%
Validation accuracy: 14.6%
Minibatch loss at step 500: 3553.364746
Minibatch accuracy: 50.0%
Validation accuracy: 44.4%
Minibatch loss at step 1000: 15120.307617
Minibatch accuracy: 60.0%
Validation accuracy: 39.1%
Minibatch loss at step 1500: 5097.240723
Minibatch accuracy: 40.0%
Validation accuracy: 40.0%
Minibatch loss at step 2000: 1844.047119
Minibatch accuracy: 30.0%
Validation accuracy: 35.2%
Minibatch loss at step 2500: 621.431091
Minibatch accuracy: 40.0%
Validation accuracy: 23.5%
Minibatch loss at step 3000: 228.831039
Minibatch accuracy: 10.0%
Validation accuracy: 27.4%
Test accuracy: 42.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [27]:
# This tutorial helps a lot: https://www.tensorflow.org/versions/v0.6.0/tutorials/mnist/tf/index.html
# PREPARATION STEP
hidden_nodes = 1024
batch_size = 300
beta = 0.002

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Set hidden layer
    hl_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hl_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hl_weights) + hl_biases)
    # Apply dropout
    hidden_layer = tf.nn.dropout(hidden_layer, 0.75)
    print(hidden_layer)
    
    # Set output layer
    ol_weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    print(ol_weights.get_shape())
    ol_biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation
    logits = tf.matmul(hidden_layer, ol_weights) + ol_biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    loss = loss + beta * (tf.nn.l2_loss(hl_weights) + tf.nn.l2_loss(ol_weights))
    
    # Optimizer - for finding the minimum of the loss using gradient descent
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)  
    
    # Predictions for the training, validation and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hl_weights) + hl_biases), ol_weights) + ol_biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hl_weights) + hl_biases),ol_weights)+ ol_biases)

Tensor("dropout/mul_1:0", shape=TensorShape([Dimension(300), Dimension(1024)]), dtype=float32)
TensorShape([Dimension(1024), Dimension(10)])


In [43]:
# EXECUTION STEP
num_steps = 10001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ('Initialized')
    for step in xrange(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
           [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1080.437012
Minibatch accuracy: 7.7%
Validation accuracy: 23.8%
Minibatch loss at step 500: 229.686081
Minibatch accuracy: 81.0%
Validation accuracy: 63.8%
Minibatch loss at step 1000: 83.883369
Minibatch accuracy: 78.7%
Validation accuracy: 62.4%
Minibatch loss at step 1500: 30.841965
Minibatch accuracy: 87.0%
Validation accuracy: 70.5%
Minibatch loss at step 2000: 11.585958
Minibatch accuracy: 89.3%
Validation accuracy: 73.3%
Minibatch loss at step 2500: 4.573484
Minibatch accuracy: 89.3%
Validation accuracy: 74.1%
Minibatch loss at step 3000: 2.043448
Minibatch accuracy: 87.3%
Validation accuracy: 74.6%
Minibatch loss at step 3500: 1.055228
Minibatch accuracy: 89.7%
Validation accuracy: 74.5%
Minibatch loss at step 4000: 0.760237
Minibatch accuracy: 88.0%
Validation accuracy: 75.2%
Minibatch loss at step 4500: 0.552427
Minibatch accuracy: 91.0%
Validation accuracy: 75.3%
Minibatch loss at step 5000: 0.492636
Minibatch accuracy: 91.0%
Validation 

---
Notes about problem 3
----------

Dropout seems to give a better result if the number of steps is greater than the "default" one I've been using throughout all the assignments (3000). As you can in this problem and in problem 2, the test accuracy is the same.
I believe that if I keep increasing the number of steps, the dropout model will perform better.

This might change if the `keep_prob` parameter changes. 

For more info check [https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#dropout](Dropout)

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [5]:
# This tutorial helps a lot: https://www.tensorflow.org/versions/v0.6.0/tutorials/mnist/tf/index.html
import math
# PREPARATION STEP

# This is also the size of the hidden layer 1
hidden_nodes = 1024
hidden_layer2_size = 300
hidden_layer3_size = 50
batch_size = 300
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Set hidden layer 1
    hl_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes],
                                                 stddev=math.sqrt(2.0/(image_size * image_size))))
    hl_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hl_weights) + hl_biases)
    #hidden_layer = tf.nn.dropout(hidden_layer, 0.75)
    print(hidden_layer)
    
    # Set hidden layer 2
    hl2_weights = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_layer2_size],
                                                  stddev=math.sqrt(2.0/hidden_nodes)))
    hl2_biases = tf.Variable(tf.zeros([hidden_layer2_size]))
    hidden_layer2 = tf.nn.relu(tf.matmul(hidden_layer, hl2_weights) + hl2_biases)
    #hidden_layer2 = tf.nn.dropout(hidden_layer2, 0.75)
    print(hidden_layer2)
    
    # Set hidden layer 3
    hl3_weights = tf.Variable(tf.truncated_normal([hidden_layer2_size, hidden_layer3_size],
                                                 stddev=math.sqrt(2.0/hidden_layer2_size)))
    hl3_biases = tf.Variable(tf.zeros([hidden_layer3_size]))
    hidden_layer3 = tf.nn.relu(tf.matmul(hidden_layer2, hl3_weights) + hl3_biases)
    #hidden_layer3 = tf.nn.dropout(hidden_layer3, 0.75)
    print(hidden_layer3)
    
    
    # Set output layer
    ol_weights = tf.Variable(tf.truncated_normal([hidden_layer3_size, num_labels],
                                                stddev=math.sqrt(2.0/hidden_layer3_size)))
    print(ol_weights.get_shape())
    ol_biases = tf.Variable(tf.zeros([num_labels]))
     
    # Training computation
    logits = tf.matmul(hidden_layer3, ol_weights) + ol_biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # L2 regularization
    loss = loss + beta * (tf.nn.l2_loss(hl_weights) + tf.nn.l2_loss(hl2_weights)
                          + tf.nn.l2_loss(hl3_weights) + tf.nn.l2_loss(ol_weights))
    
    '''
    Optimizer - for finding the minimum of the loss using gradient descent.
    Exponential decay is also applied here
    '''
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(learning_rate=0.5, global_step=global_step,
                                               decay_steps=100, decay_rate=0.86, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training, validation and test data
    train_prediction = tf.nn.softmax(logits)
    
    
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(
                                tf_valid_dataset, hl_weights) + hl_biases), hl2_weights) + hl2_biases),
                                      hl3_weights) + hl3_biases), ol_weights) + ol_biases)
    
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(
                                tf_test_dataset, hl_weights) + hl_biases), hl2_weights) + hl2_biases),
                                      hl3_weights) + hl3_biases), ol_weights) + ol_biases)

Tensor("Relu:0", shape=TensorShape([Dimension(300), Dimension(1024)]), dtype=float32)
Tensor("Relu_1:0", shape=TensorShape([Dimension(300), Dimension(300)]), dtype=float32)
Tensor("Relu_2:0", shape=TensorShape([Dimension(300), Dimension(50)]), dtype=float32)
TensorShape([Dimension(50), Dimension(10)])


In [6]:
# EXECUTION STEP
num_steps = 15001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ('Initialized')
    for step in xrange(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
           [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.345499
Minibatch accuracy: 13.3%
Validation accuracy: 26.9%
Minibatch loss at step 500: 1.160205
Minibatch accuracy: 87.7%
Validation accuracy: 73.8%
Minibatch loss at step 1000: 1.072827
Minibatch accuracy: 88.0%
Validation accuracy: 75.9%
Minibatch loss at step 1500: 1.017812
Minibatch accuracy: 86.7%
Validation accuracy: 76.1%
Minibatch loss at step 2000: 0.866149
Minibatch accuracy: 93.0%
Validation accuracy: 76.5%
Minibatch loss at step 2500: 0.872486
Minibatch accuracy: 91.0%
Validation accuracy: 76.7%
Minibatch loss at step 3000: 0.895210
Minibatch accuracy: 91.7%
Validation accuracy: 76.8%
Minibatch loss at step 3500: 0.839318
Minibatch accuracy: 91.7%
Validation accuracy: 76.8%
Minibatch loss at step 4000: 0.896604
Minibatch accuracy: 92.0%
Validation accuracy: 76.8%
Minibatch loss at step 4500: 0.849578
Minibatch accuracy: 92.7%
Validation accuracy: 76.8%
Minibatch loss at step 5000: 0.839932
Minibatch accuracy: 92.3%
Validation accurac